# 타율 예측

**직전 50경기 타율들과 평균타율, 승률, 방어율로 이후 20경기 예측하기

    train set : 14-63 / 24-83 / 54-103 번째 경기
    test set  : 74-123 번째 경기

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. 데이터 가져오기

In [16]:
t_bat16=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv",encoding='utf-8',engine='python')
t_bat17=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv",encoding='utf-8',engine='python')
t_bat18=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv",encoding='utf-8',engine='python')
t_bat19=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv",encoding='utf-8',engine='python')

In [17]:
t_pit16=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv",encoding='utf-8',engine='python')
t_pit17=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv",encoding='utf-8',engine='python')
t_pit18=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv",encoding='utf-8',engine='python')
t_pit19=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv",encoding='utf-8',engine='python')

In [18]:
#모든 열 다 출력하기
pd.options.display.max_columns=None

## 2. train test 셋 만들기

In [20]:
team=['HH', 'HT', 'KT', 'LG', 'LT', 'NC', 'OB', 'SK', 'SS', 'WO']

#### 승률 변수 만들기

In [22]:
pd.set_option('mode.chained_assignment',None) #chain copy warning 제거하기

def make_win_lose(data):
    data['W']=0; data['L']=0
    data['W'][data['WLS']=='W']=1
    data['L'][data['WLS']=='L']=1
    return data

In [23]:
#팀투수 데이터에 승,패 여부 추가
data_list=[t_pit16,t_pit17,t_pit18,t_pit19] 
for data in data_list:
    data=make_win_lose(data)

In [24]:
#팀타자 데이터에 승,패 변수 추가
t_bat16=pd.merge(t_bat16,t_pit16[['G_ID','T_ID','W','L','ER','INN2']],on=['G_ID','T_ID'])
t_bat17=pd.merge(t_bat17,t_pit17[['G_ID','T_ID','W','L','ER','INN2']],on=['G_ID','T_ID'])
t_bat18=pd.merge(t_bat18,t_pit18[['G_ID','T_ID','W','L','ER','INN2']],on=['G_ID','T_ID'])
t_bat19=pd.merge(t_bat19,t_pit19[['G_ID','T_ID','W','L','ER','INN2']],on=['G_ID','T_ID'])

#### 타율 변수 만드는 함수

In [25]:
def make_data(train_data,answer_data):
    train_data['HIT_RATE']=train_data['HIT']/train_data['AB']
    tr=train_data[['T_ID','HIT_RATE']]                          #경기당 팀과 타율만 남김
    num=list(range(1,int((len(train_data)/10)+1)))*10           #팀별 경기순서 추가
    tr=tr.assign(order=num)
    tr=tr.pivot(index='T_ID',columns='order',values='HIT_RATE') #팀당 경기별 타율 데이터셋 만들기
    
    data=pd.merge(tr,answer_data,on='T_ID')                     #팀별 앞124경기와 후20경기 결과 합치기
    return data

#### train, test set 만드는 함수

In [54]:
t_bat=[t_bat16,t_bat17,t_bat18,t_bat19]
def make_answer_train(year):
    a=t_bat[year-2016][['T_ID','HIT','AB','W','L','ER','INN2','PA','RBI', 'RUN', 'H2', 'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB',
                       'IB', 'HP', 'KK', 'GD', 'ERR', 'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN']]  
    data=pd.DataFrame()  #빈 데이터프레임 만들기
    pa=pd.DataFrame()    #빈 데이터프레임 만들기

    train_data=[pd.DataFrame() for var in range(1,5)]   #50경기씩 묶일 데이터프레임
    answer=[pd.DataFrame() for var in range(1,5)]       #최종 후반 20경기 묶일 데이터프레임
        
    for i in range(4):
        tra=train_data[i]
        ans=answer[i]
        for t in team:
            tra=pd.concat([tra,a[a['T_ID']==t][14+i*20:14+50+i*20]])     #각 팀별 앞 14경기 제외하고, 50경기씩 묶음(train set)
            ans=pd.concat([ans,a[a['T_ID']==t][14+50+i*20:14+70+i*20]])  #train set의 경기의 다음 20경기 묶음(test set)
    
        #팀별 AB,HIT합 구하기
        ans=ans.groupby(['T_ID']).sum()
        ans['HIT_RATE']=ans['HIT']/ans['AB']              #타율 계산
        ans=ans.drop(ans.columns[list(range(26))],axis='columns')
        
        #변수 생성
        tr=tra.groupby(['T_ID']).sum() 
        tr['WINNING_RATE'] = tr['W']/(tr['W']+tr['L'])    #승률 변수생성
        tr['ERA'] = tr['ER']*27/tr['INN2']                #방어율 변수생성
        
        #변수 삭제
        tr=tr.drop(['HIT','W','L','ER','INN2'],axis='columns')
        
        if i <3:
            data=pd.concat([data,make_data(tra,ans)])  
            pa=pd.concat([pa,tr])
        else:
            test_data=make_data(tra,ans)
            test_data['WINNING_RATE']=tr['WINNING_RATE']   #test set에 승률 추가
            test_data['ERA']=tr['ERA']                     #test set에 ERA 추가
            
    #변수 추가
    data['MEAN']=data.iloc[:,:50].mean(axis=1)             #train set에 평균 추가
    test_data['MEAN']=test_data.iloc[:,:50].mean(axis=1)   #test set 에 평균 추가
    data=pd.concat([data,pa],axis=1)                                 #train set에 ERA 추가
        
    return data,test_data

In [55]:
#train, test set 만들기
train16,test16=make_answer_train(2016)
train17,test17=make_answer_train(2017)
train18,test18=make_answer_train(2018)
train19,test19=make_answer_train(2019)

In [59]:
aaa=train16.drop(train16.columns[list(range(50))],axis='columns')

In [61]:
aaa.corr()

,HIT_RATE,MEAN,AB,PA,RBI,RUN,H2,H3,HR,SB,CS,SH,SF,BB,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,WINNING_RATE,ERA
HIT_RATE,1.000000,-0.011735,-0.257567,-0.289146,-0.156732,-0.137762,-0.226555,0.356330,0.021075,-0.031633,0.044922,0.057628,-0.041627,-0.226680,0.094474,0.074001,-0.047337,-0.075262,0.192420,-0.246252,0.029728,-0.264156,-0.208547,-0.119228,-0.102559
MEAN,-0.011735,1.000000,0.636561,0.631424,0.817104,0.807679,0.705140,0.585133,0.501101,-0.003508,0.110048,-0.212396,0.292764,0.208103,0.431781,0.467140,-0.047954,0.037106,-0.383658,0.327834,0.660458,0.637880,0.690736,0.618618,-0.569630
AB,-0.257567,0.636561,1.000000,0.775095,0.577450,0.537469,0.529567,0.243960,0.290460,0.016222,-0.046810,-0.020482,-0.145522,-0.015364,0.174217,0.456008,0.223227,-0.342537,-0.332612,0.518271,0.193390,0.554524,0.350035,0.129529,-0.149025
PA,-0.289146,0.631424,0.775095,1.000000,0.766565,0.762600,0.634492,0.170025,0.328718,-0.006791,-0.231868,0.253267,0.263686,0.557851,0.477239,0.574879,-0.055566,-0.018679,-0.591766,0.759113,0.408930,0.755902,0.567580,0.386299,-0.123066
RBI,-0.156732,0.817104,0.577450,0.766565,1.000000,0.985988,0.687289,0.410692,0.646870,-0.236986,-0.317200,-0.085956,0.349862,0.500151,0.490887,0.481511,-0.090410,0.182774,-0.542070,0.323269,0.779781,0.639194,0.807278,0.819798,-0.523989
RUN,-0.137762,0.807679,0.537469,0.762600,0.985988,1.000000,0.725662,0.417310,0.616345,-0.174466,-0.320541,-0.079399,0.421910,0.536261,0.486244,0.482398,-0.156946,0.198845,-0.574878,0.305768,0.785046,0.641312,0.804552,0.813144,-0.503394
H2,-0.226555,0.705140,0.529567,0.634492,0.687289,0.725662,1.000000,0.307222,0.312549,0.213969,0.034031,-0.153090,0.332577,0.356611,0.433852,0.410411,-0.126559,-0.022551,-0.495435,0.382282,0.492344,0.686774,0.606212,0.473989,-0.259427
H3,0.356330,0.585133,0.243960,0.170025,0.410692,0.417310,0.307222,1.000000,0.177927,-0.026647,0.084973,-0.159262,0.114205,-0.029648,0.179627,0.166304,0.007477,-0.007078,-0.304613,-0.013798,0.579951,0.321051,0.462891,0.330253,-0.430844
HR,0.021075,0.501101,0.290460,0.328718,0.646870,0.616345,0.312549,0.177927,1.000000,-0.546726,-0.346738,-0.022541,0.277605,0.007565,0.373188,0.602832,0.066689,0.147455,-0.150271,-0.009742,0.308097,-0.013516,0.221651,0.529382,-0.582962
SB,-0.031633,-0.003508,0.016222,-0.006791,-0.236986,-0.174466,0.213969,-0.026647,-0.546726,1.000000,0.403896,-0.247652,-0.096740,0.131517,0.009195,-0.256161,0.079128,-0.092118,0.070957,0.153883,0.004985,0.266999,0.083863,-0.258643,0.348287


## 3. 모델학습(Random Forest)

In [12]:
# 모델 모듈
import xgboost
from xgboost import XGBRegressor, plot_importance
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [21]:
#### 학습시킬 모델
model_dict = {'xgb':XGBRegressor(n_estimators=100, learning_rate=0.05 ,max_depth=3, random_state=23),
    'lgbm':LGBMRegressor(),
    'rf':RandomForestRegressor(random_state=1234),
    'svr':SVR(),
    'reg':LinearRegression(),
    'ridge':Ridge(alpha=100,normalize=True),
    'lasso':Lasso(alpha=0.05,normalize=True)}  #변수 자동 채택 / 많은 변수를 다룰때 활용

In [22]:
X_cols=list(range(1,51))+['MEAN','WINNING_RATE','ERA']
Y_col=['HIT_RATE']

In [29]:
train=train16
test=test16

In [49]:
train=pd.read_csv("C:/Users/a/Desktop/2020빅콘/데이터/정석 데이터.csv",encoding='utf-8',engine='python')[['ERA','HITRATE','OPS.Af','SLG.Af','ERA.Af']] 
test=pd.read_csv("C:/Users/a/Desktop/2020빅콘/데이터/정석데이터2.csv",encoding='utf-8',engine='python')[['ERA','HITRATE','OPS.Af','SLG.Af','ERA.Af']] 

In [52]:
X_cols=['ERA','HITRATE','OPS.Af','SLG.Af']
Y_col=['ERA.Af']

In [58]:
X_train = train[X_cols]; Y_train = train[Y_col]
X_test = test[X_cols]; Y_test = test[Y_col]
diff=[]

#모델 여러개 돌려보기
model_list=['xgb','lgbm','rf','svr','reg','ridge','lasso']

for m in model_list:
    model = model_dict[m]
    if m=='rf': Y_train=Y_train.values.ravel()  #rf에서의 오류 없애기 위함
    model.fit(X_train, Y_train)  
    Y_predict = model.predict(X_test)

    r2=r2_score(Y_test, Y_predict)

    a=Y_test.assign(predict=Y_predict)
    a['diff']=a['ERA.Af']-a['predict']
    print(m,"\t2016 :",abs(a['diff'][:10]).sum(),"\t2017 :",abs(a['diff'][10:20]).sum(),"\t2018 :",abs(a['diff'][20:30]).sum(),"\t2019 :",abs(a['diff'][30:]).sum())
    diff.append(r2)

xgb 	2016 : 7.631818586989018 	2017 : 6.871333612852353 	2018 : 3.926584522874286 	2019 : 7.015231322855829
lgbm 	2016 : 7.808917577756286 	2017 : 7.134752141687321 	2018 : 4.117122112205005 	2019 : 6.329534344951519
rf 	2016 : 5.496918006948373 	2017 : 5.8542211588301205 	2018 : 3.1322415236286894 	2019 : 5.670983995895508
svr 	2016 : 6.419003129688526 	2017 : 6.000978569549434 	2018 : 4.860573483365992 	2019 : 14.353030616232024
reg 	2016 : 7.853811374713206 	2017 : 6.522985140397103 	2018 : 3.8556299931895026 	2019 : 9.877363719243885
ridge 	2016 : 7.089376140339368 	2017 : 6.999268901105118 	2018 : 5.820302193071125 	2019 : 16.223097547651985
lasso 	2016 : 7.12144377655775 	2017 : 7.02687837069537 	2018 : 5.870156838110946 	2019 : 16.37466962673563


### 예측값과 실제값 차이

In [45]:
print('2016 >',abs(result[0]['diff']).sum())
result[0]

NameError: name 'result' is not defined

In [17]:
print('2017 >',abs(result[1]['diff']).sum())
result[1]

2017 > 0.11828556352453623


,HIT_RATE,predict,diff
T_ID,,,
HH,0.275568,0.294748,-0.019180
HT,0.298295,0.295506,0.002789
KT,0.297486,0.288253,0.009233
LG,0.271533,0.282038,-0.010505
LT,0.296296,0.291901,0.004395
NC,0.331579,0.291498,0.040081
OB,0.295105,0.288795,0.006310
SK,0.297450,0.295501,0.001949
SS,0.290323,0.286012,0.004311


In [18]:
print('2018 >',abs(result[2]['diff']).sum())
result[2]

2018 > 0.12427250371235726


,HIT_RATE,predict,diff
T_ID,,,
HH,0.276878,0.283936,-0.007058
HT,0.279830,0.287772,-0.007942
KT,0.287879,0.293752,-0.005874
LG,0.267908,0.286285,-0.018377
LT,0.300699,0.288046,0.012653
NC,0.271408,0.283710,-0.012302
OB,0.315646,0.300869,0.014778
SK,0.281471,0.291455,-0.009984
SS,0.306207,0.296996,0.009211


In [19]:
print('2019 >',abs(result[3]['diff']).sum())
result[3]

2019 > 0.2252243315141516


,HIT_RATE,predict,diff
T_ID,,,
HH,0.266764,0.264463,0.002301
HT,0.253373,0.268567,-0.015194
KT,0.272321,0.267121,0.005200
LG,0.257485,0.273886,-0.016401
LT,0.220877,0.271000,-0.050123
NC,0.275109,0.267633,0.007476
OB,0.281690,0.269475,0.012215
SK,0.239695,0.290985,-0.051290
SS,0.232061,0.277658,-0.045596
